In [ ]:
# 크롤링 순서
# 1차 크롤링 : 블로그 조회 -> title / url / date
# 2차 크롤링 : url별 방문하여 해시태그 가져오기


- import

In [2]:
import os
import sys
import pandas as pd

import time
from datetime import datetime, date, timedelta
from bs4 import BeautifulSoup

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

- url list 

In [3]:
# start_time = timer()

# 검색어 입력
f = open("data/station.txt", 'r', encoding='utf-8')
keywords = f.readlines()
keywords_data = []

# 조회 기간 설정 : 7일(days에서 수정 가능)
# startDate=7일전 날짜, endDate=오늘 날짜
date = datetime.now()
startDate= (date+timedelta(days=-7)).strftime('%Y-%m-%d')
endDate = (datetime.now()).strftime('%Y-%m-%d')

# 브라우저 옵션 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36')
# 브라우저 초기화
driver = webdriver.Chrome(options=options)

crawling_blog_data = []
for query in keywords:
    # start_crawling_time = timer()
    # print(start_crawling_time)
    try:
        # 검색 URL 생성
        base_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
        driver.get(base_url)
        time.sleep(0.5)  # 페이지가 로드될 때까지 대기

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        search_number = soup.find(class_="search_number")
        total_num = search_number.text.replace("건",'').replace(",",'')
        total_num = int(total_num)
        end_page=total_num//7+1

        # 페이지마다 게시물 가져오기
        for page_num in range(1, end_page+1):
            search_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
            driver.get(search_url)
            time.sleep(0.5)  # 페이지가 로드될 때까지 대기

            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            # 블로그 포스트 데이터 가져오기
            posts_data = soup.select('div.info_post > div.desc > a.desc_inner')
            posts_date_data = soup.select('div.info_post > div.writer_info')

            # 타이틀 수가 7개가 아니면 루프를 빠져나오기
            if len(posts_data) != 1:
                for post, post_date in zip(posts_data, posts_date_data):
                    title = post.text.replace('\n', '').strip()
                    href = post.attrs['href']
                    date = post_date.find('span', 'date').text.strip()
                    try :
                        date_timestamp = datetime.strptime(date, '%Y. %m. %d.').strftime('%Y-%m-%d')
                    except:
                        date_timestamp = endDate
                    crawling_blog_data.append({"location": query, "title": title, "url": href, "posted_date": date_timestamp})
            else :
                break
    except Exception as e:
        print(f"에러 발생: {e}")

    # end_crawling_time = timer()
    # print(end_crawling_time)
    # print("working_crawling_time: {} sec".format(end_crawling_time-start_crawling_time))


# 브라우저 종료
driver.quit()
# end_time = timer()
# print("working_time: {} sec".format(end_time-start_time))


crawling_blog_data = pd.DataFrame(crawling_blog_data)
file_name = f'data/crawling_blog_list_{startDate}-{endDate}.csv'
crawling_blog_data.to_csv(file_name, index=False)

- hashtag list

In [26]:
# url list 가져오기
crawling_data = pd.read_csv(file_name)
crawling_data

# Chrome driver 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome(options = options)


crawling_contents_data = []
url_lst = crawling_data['url']
for i in range(len(url_lst)) :
    url = str(url_lst[i])
    driver.get(url)

    title = crawling_data.loc[i,'title']
    date = crawling_data.loc[i,'posted_date']
    try :
        # iframe 접근
        WebDriverWait(driver,10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, 'mainFrame')))

        # tag 찾아서 가져오기
        tags_element = driver.find_element(By.CLASS_NAME, "wrap_tag")
        tags = tags_element.text.strip('태그').replace('\n','')

        # data 담기
        crawling_contents_data.append({"location":query, "title":title, "url":url, "tags":tags, "posted_date":date})
    except Exception as e:
        print(f"error ({i}) : {str(e)}")

driver.quit()

In [ ]:
# 확인
print(len(crawling_blog_data))
print(len(crawling_contents_data))

- 파일저장

In [ ]:
df_tags = pd.DataFrame(crawling_contents_data)
df_tags

In [30]:
file_name = f'data/crawling_blog_data_{startDate}-{endDate}.csv'
df_tags.to_csv(f".csv", index=False)

## test

In [10]:
# start_time = timer()

# 검색어 입력
# f = open("crawling/station.txt", 'r', encoding='utf-8')
# keywords = f.readlines()
# keywords_data = []
query= input()

# 조회 기간 설정 : 7일(days에서 수정 가능)
# startDate=7일전 날짜, endDate=오늘 날짜
date = datetime.now()
startDate= (date+timedelta(days=-7)).strftime('%Y-%m-%d')
endDate = (datetime.now()).strftime('%Y-%m-%d')

# 브라우저 옵션 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36')
# 브라우저 초기화
driver = webdriver.Chrome(options=options)

crawling_blog_data = []

# start_crawling_time = timer()
# print(start_crawling_time)
try:
    # 검색 URL 생성
    base_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
    driver.get(base_url)
    time.sleep(0.5)  # 페이지가 로드될 때까지 대기

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    search_number = soup.find(class_="search_number")
    total_num = search_number.text.replace("건",'').replace(",",'')
    total_num = int(total_num)
    end_page=total_num//7+1

    # 페이지마다 게시물 가져오기
    for page_num in range(1,end_page+1):
        search_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
        driver.get(search_url)
        time.sleep(0.5)  # 페이지가 로드될 때까지 대기

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # 블로그 포스트 데이터 가져오기
        posts_data = soup.select('div.info_post > div.desc > a.desc_inner')
        posts_date_data = soup.select('div.info_post > div.writer_info')

        # 타이틀 수가 7개가 아니면 루프를 빠져나오기
        if len(posts_data) != 1 :
            for post, post_date in zip(posts_data, posts_date_data):
                title = post.text.replace('\n', '').strip()
                href = post.attrs['href']
                date = post_date.find('span', 'date').text.strip()
                try :
                    date_timestamp = datetime.strptime(date, '%Y. %m. %d.').strftime('%Y-%m-%d')
                except:
                    date_timestamp = endDate
                crawling_blog_data.append({"location": query, "title": title, "url": href, "posted_date": date_timestamp})
        else :
            break
except Exception as e:
    print(f"에러 발생: {e}")

# end_crawling_time = timer()
# print(end_crawling_time)
# print("working_crawling_time: {} sec".format(end_crawling_time-start_crawling_time))


# 브라우저 종료
driver.quit()
# end_time = timer()
# print("working_time: {} sec".format(end_time-start_time))

In [11]:
end_page
startDate
endDate

'2024-05-26'

In [12]:
df = pd.DataFrame(crawling_blog_data)
df.sort_values("posted_date", axis = 0, ascending=False, inplace=True)
file_name = f"data/naver_blog_list_{startDate}-{endDate}.csv"
df.to_csv(file_name, index=False)

In [13]:
blog_lst_data = pd.read_csv(file_name)
blog_lst_data

,location,title,url,posted_date
0,강남역,나혼산에 출연했던 수시로 생각나게 만드는 강남역 맛집 빠레뜨한남 강남유퀴즈에서...,https://blog.naver.com/bray9013/223458014219,2024-05-26
1,강남역,"역삼역 ""마노디세프"" 강남역점",https://blog.naver.com/janekorea69/223458581478,2024-05-26
2,강남역,역삼 강남 무한리필 맛집 강남역 고깃집 추천 달빛화로,https://blog.naver.com/duck097/223458715282,2024-05-26
3,강남역,강남역미용실 잘하는 곳 찾으신다면 슬론헤어 강추해요,https://blog.naver.com/s2s2s_73/223458313431,2024-05-26
4,강남역,[강남역 토익 저녁] 제니토익과 함께 2개월만에 토익 고득점 졸업!,https://blog.naver.com/hahahahaa2/223458698808,2024-05-26
...,...,...,...,...
3995,강남역,강남역 맛집 직접 구워주는 고깃집열 논현직영점으로 회식하러 갔다가...,https://blog.naver.com/sco0839/223451704153,2024-05-19
3996,강남역,'강남역 일부 침수' 반복…이유는 삼성전자 때문?,https://blog.naver.com/reach8011/223451346119,2024-05-19
3997,강남역,강남역 근처카페 커피 디저트 인테리어 모두 다 100점,https://blog.naver.com/weperfect/223451271679,2024-05-19
3998,강남역,전체 임플란트가격 및 강남역임플란트 분석,https://blog.naver.com/hlappy/223451422542,2024-05-19


In [14]:
# Chrome driver 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome(options = options)


crawling_contents_data = []

url_lst = blog_lst_data['url']
for i in range(len(url_lst)) :
    url = str(url_lst[i])
    driver.get(url)

    try :
        # iframe 접근
        WebDriverWait(driver,10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, 'mainFrame')))

        # tag 찾아서 가져오기
        tags_element = driver.find_element(By.CLASS_NAME, "wrap_tag")
        tags = tags_element.text.strip('태그').replace('\n','')
        # overlays_tag = "wrap_tag"
        # html_tags = driver.find_element(By.CSS_SELECTOR, overlays_tag)
        # hash_tags = html_tags.text
        crawling_contents_data.append({"location":query, "url":url, "tags":tags})
    except Exception as e:
        print("error ({i}) : {str(e)}")

driver.quit()


error ({i}) : {str(e)}


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=125.0.6422.77)
Stacktrace:
	GetHandleVerifier [0x00007FF7A9DD1F22+60322]
	(No symbol) [0x00007FF7A9D4CE99]
	(No symbol) [0x00007FF7A9C07EBA]
	(No symbol) [0x00007FF7A9BEF1CC]
	(No symbol) [0x00007FF7A9BEF090]
	(No symbol) [0x00007FF7A9C0A4E1]
	(No symbol) [0x00007FF7A9C9B359]
	(No symbol) [0x00007FF7A9C7BFC3]
	(No symbol) [0x00007FF7A9C49617]
	(No symbol) [0x00007FF7A9C4A211]
	GetHandleVerifier [0x00007FF7AA0E946D+3301613]
	GetHandleVerifier [0x00007FF7AA133693+3605267]
	GetHandleVerifier [0x00007FF7AA129410+3563664]
	GetHandleVerifier [0x00007FF7A9E842F6+790390]
	(No symbol) [0x00007FF7A9D574DF]
	(No symbol) [0x00007FF7A9D533D4]
	(No symbol) [0x00007FF7A9D53562]
	(No symbol) [0x00007FF7A9D42F6F]
	BaseThreadInitThunk [0x00007FFB6884257D+29]
	RtlUserThreadStart [0x00007FFB69CCAA48+40]


In [15]:
print(len(blog_lst_data))
print(len(crawling_contents_data))

4000
3842


In [16]:
df_tags = pd.DataFrame(crawling_contents_data)
df_tags

,location,url,tags
0,강남역,https://blog.naver.com/bray9013/223458014219,#빠레뜨한남#강남역맛집#강남역점심#강남역파스타#강남역데이트#강남역모임
1,강남역,https://blog.naver.com/janekorea69/223458581478,#강남맛집#강남회식#강남모임#강남데이트#역삼맛집#역삼역양식#역삼역회식#역삼역데이트#...
2,강남역,https://blog.naver.com/duck097/223458715282,#강남고기무한리필#강남무한리필맛집#역삼회식#달빛화로#강남무한리필고기
3,강남역,https://blog.naver.com/s2s2s_73/223458313431,#슬론헤어#강남미용실#강남헤어샵#서초구미용실#강남역미용실
4,강남역,https://blog.naver.com/hahahahaa2/223458698808,#강남역토익저녁#강남역토익#강남역토익학원#파고다어학원#파고다어학원강남#파고다강남#강...
...,...,...,...
3837,강남역,https://blog.naver.com/pr0501/223434697529,#강남역맛집#강남역고기집#강남역삼겹살#강남역냉삼#강남역회식#강남역냉동삼겹살
3838,강남역,https://blog.naver.com/nnnnnan2/223451017259,#동남아시아음식#강남태국음식#강남역베트남음식#강남역태국음식#강남역쌀국수#강남역푸팟퐁...
3839,강남역,https://blog.naver.com/ummi0518/223451539857,#하우마라탕#강남역마라탕#강남역식사#강남역점심
3840,강남역,https://blog.naver.com/choochooyaa/223451805873,#성시경먹을텐데#역삼강남진해장#강남진해장#강남역12번출구#내돈내산#곱창전골#강남역맛...


In [17]:
df_tags.to_csv("tags_testg(회기역).csv", index=False)